# Normalize the data

- dates
- split products
- split ammounts
- split prices
- prices with coma

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("../data/inital_dataset.csv")

In [14]:
# Dates
df['Fecha'] = pd.to_datetime(df['Fecha'], format='mixed', dayfirst=True)